In [1]:
from keras.datasets import mnist
import numpy as np
import pandas as pd
import sys
import copy
import math
from sklearn.preprocessing import normalize

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape([60000,28*28])
x_train.shape

Using TensorFlow backend.


(60000, 784)

In [2]:

n_sample=300
X=x_train[:n_sample,:].T #traspuesta, donde las observaciones son columnas
X.shape

(784, 300)

In [3]:
# Normalizamos los datos:
X = normalize(X - np.outer(np.mean(X, axis=1), np.ones(X.shape[1])), axis=0)
X.shape

(784, 300)

A partir de las celdas de abajo se definirá una variación de la función SpectralClustering de sklearn, para logar rescatar el *manifold* que contiene el ebedding inicial con *eigenvectors*

In [0]:
import numpy as np
import sys
import copy

class MatchingPursuit:
	"""Simple implementation of the Matching Pursuit (MP) algorithm
	
  FUENTE: https://github.com/mitscha/ssc_mps_py/blob/master/matchingpursuit.py
  
	Parameters
	----------
	
	smax: int
		Maximum number of MP iterations
	pmax: int, optional
		Maximum sparsity level of x (default: pmax = smax)
	tol: float, optional
		Threshold on approximation quality ||Ax - y|| (default: 0.0)
		
	Attributes
	----------
	
	coef_: array, shape (n_samples)
		coefficient vector (solution)
	
	
	Note: Stops after smax iterations, or when approximation quality specified by tol
	      is attained, or when the sparsity level of the coefficient vector is pmax
	
	"""
	
	def __init__(self,smax,pmax=None,tol=None):
		self._smax = smax if smax != None else sys.maxsize
		self._pmax = pmax if pmax != None else smax
		self._tol = 0.0 if tol == None else tol
		self.coef_ = None
	
	def fit(self,A,y):
		"""
		Finds a sparse (approximate) solution x to Ax = y
		
		Parameters
		----------
		
		X: dictionary, array, shape (n_features, n_samples)
		y: target, array, shape (n_features)
		
		"""
		
		assert(len(A.shape) == 2)
		assert(len(y.shape) == 1 and A.shape[0] == y.shape[0])
		
		x = np.zeros(A.shape[1])
		r = copy.deepcopy(y)
		nit = 0
		while np.linalg.norm(r) > self._tol \
			and nit < self._smax \
			and np.sum(np.abs(x) > 0) < self._pmax:
			
			idx = np.argmax(np.dot(r.T,A))
			dx = np.dot(r.T,A[:,idx])/np.dot(A[:,idx].T,A[:,idx])
			x[idx] += dx
			r -= dx*A[:,idx]
			nit += 1
		
		self.coef_ = x

In [0]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.cluster import SpectralClustering
from sklearn.manifold import SpectralEmbedding
from sklearn.preprocessing import normalize

def ssc_mps_modificado(X,smax,L,tol=None,alg_name='OMP',pmax=None):
	"""
  Esta es una versión modificada de la función 'ssc_mps', implementada por
  https://github.com/mitscha/ssc_mps_py/blob/master/ssc_mps.py
  
	Implements Sparse Subspace Clustering-Orthogonal Matching Pursuit (SSC-OMP) and 
	SSC-Matching Pursuit (SSC-MP)
	
	Parameters
	----------
	
	X: array, shape (n_features, n_samples)
		data matrix
	smax: int
		Maximum number of OMP/MP iterations
	L: int
		Number of clusters
	tol: float, optional
		Threshold on approximation quality
	alg_name: string, optional
		'OMP' (default) or 'MP'
	pmax:
		Maximum sparsity level for MP
	
	
	Note: 
	
	- Stopping behavior:
	  SSC-OMP: Stop after smax iterations if tol=None, stop when approximation quality
	           specified by tol is attained otherwise
	  SSC-MP:  Stop after smax iterations, or when approximation quality specified by tol
	           is attained, or when the sparsity level of the coefficient vector is pmax
	- See https://arxiv.org/abs/1612.03450 for a discussion of the stopping criteria
	
	"""
	
	
	XX = np.array(X)
	
	assert(len(XX.shape) == 2)
	
	m = XX.shape[0]
	N = XX.shape[1]
	
	
	alg = None
	if alg_name == 'MP':
		alg = MatchingPursuit(smax, pmax, tol)
	else:
		alg = OrthogonalMatchingPursuit(
			n_nonzero_coefs=smax, 
			tol=tol, 
			fit_intercept=False, 
			normalize=False)
	
	
	C = np.zeros((N,N))
	
	for i in range(N):
		data_idx = [j for j in range(i)]
		data_idx.extend([j for j in range(i+1,N)])
		alg.fit(XX[:,data_idx],np.squeeze(XX[:,i]))
		c = np.zeros(N)
		c[data_idx] = alg.coef_
		C[:,i] = c
	maps=SpectralEmbedding(affinity='precomputed', n_components=L, eigen_solver='arpack').fit(np.abs(C)+np.abs(C.T))
  # Se utiliza ahora una descomposición por eigenvectors
	sc = SpectralClustering(n_clusters=L, affinity='precomputed', n_init=50, n_jobs=-1, eigen_solver='arpack')
	sc.fit(np.abs(C) + np.abs(C.T)) # Se introduce la matriz de afinidad descrita en SUB-GAN
	
	return sc.labels_, maps.embedding_

In [0]:
K = 10 # Numero de grupos
soft_assign, eigen_embedding=ssc_mps_modificado(X,K,K)

In [7]:
# Verificamos que se obtienen las asignaciones correctas y el embedding tras la descomposición:
soft_assign.shape, eigen_embedding.shape

((300,), (300, 10))

In [8]:
# Este es el resultado del SpectralClustering utilizando la descomposición por
# eigenvectors con el número equivalente de dimensiones = 10
pd.crosstab(soft_assign,y_train[:n_sample]) 

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,0,8,12,9,2,3,1,2,1,2
1,21,0,0,0,1,0,2,0,0,0
2,0,3,0,0,0,0,0,0,0,0
3,1,0,1,0,13,0,0,8,1,21
4,0,0,0,0,0,0,0,15,0,0
5,7,10,15,7,16,4,5,4,13,7
6,0,18,0,0,0,0,0,0,0,0
7,0,0,0,0,0,13,0,0,0,0
8,0,0,0,18,0,3,0,0,6,1
